# Assignment 1 – Glacier Mass Balance Model

Let's go!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Nr 1 – calculate the mass balance 1980-2009

## Nr 2 - choose and read in climate data

Possible stations: 
 - Jungfraujoch (3571 MSL)
 - Grimsel Hospiz (1980 MSL)
 - how high is Fieschergletscher?
 
Norm value charts for Precipitation:
 - Precip 61-90, precip 91-20 and sunshine 91-20
 - Yellow left Jungfraujoch, yellow right Grimsel, orange ~ location of Fieschergletscher
 - Glacier in the area with highest precipitation of switzerland, Jungfraujoch gets much less (even though its in the same glaciated area), Grimsel might be better?
 - both stations in data_raw folder
 - URL for data: https://www.meteoswiss.admin.ch/services-and-publications/applications/ext/climate-tables-homogenized.html 
 - URL if you want to look at 
 
<div>
<img src="maps/precip_61-90.png" width="700" align="left"/>
<img src="maps/precip_91-20.png" width="400" align="left"/>
<img src="maps/sunshine_91-20.png" width="400" align="left"/>
</div>
 
 